## <u> Demonstrate some generic techniques for sonifying 1D data:</u>

**First, import relevant modules:**

In [ ]:
%reload_ext autoreload 
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
from strauss.sonification import Sonification
from strauss.sources import Objects
from strauss import channels
from strauss.score import Score
from strauss.generator import Synthesizer
import IPython.display as ipd
import os
from scipy.interpolate import interp1d
import numpy as np
import requests
import json

**Now, we load some test data!**

We use a simulated subject from the PH-TESS dataset (subject 94880899 or subject 94880882):

In [ ]:
s94812564 = 'https://panoptes-uploads.zooniverse.org/subject_location/c4a2a54a-da3b-4f1d-9823-702549a2806a.txt'
s94880899 = 'https://panoptes-uploads.zooniverse.org/subject_location/03caefe6-7d48-4225-9cc0-e5a60f337af8.txt'
s94880882 = 'https://panoptes-uploads.zooniverse.org/subject_location/7578bd5d-b8da-4c42-8da7-5f897ded018e.txt'
response = json.loads(requests.get(s94812564).text)
days = np.array(response['x'])
flux = np.array(response['y'])
normalised_days = lambda d: d / days.max()
x = np.array(list(map(normalised_days, days)))
normalised_flux = lambda f: ((f - flux.min()) / (flux.max() - flux.min()))
y = np.array(list(map(normalised_flux, flux)))


plt.plot(days,y)
plt.ylabel('Normalised Flux')
plt.xlabel('Days')


**Set up some universal sonification parameters and classes for the examples below**

For all examples we use the `Synthesizer` generator to create a 30 second, mono sonification.

In [ ]:
# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "stereo"

# length of the sonification in s
length = 15.

# set up synth and turn on LP filter
generator = Synthesizer()
generator.load_preset('pitch_mapper')
generator.preset_details('pitch_mapper')

### <u>Example 1</u> &nbsp; **Pitch Mapping**

In [ ]:
notes = [["A2"]]
score =  Score(notes, length)

data = {'pitch':1.,
        'time_evo':x,
        'azimuth':(x*0.5+0.25) % 1,
        'polar':0.5,
        'pitch_shift':y**0.7}

# set up source
sources = Objects(data.keys())
sources.fromdict(data)
sources.apply_mapping_functions()

soni = Sonification(score, sources, generator, system)
soni.render()
soni.notebook_display()

### <u>Example 2</u> &nbsp; **Volume Mapping**

In [ ]:
notes = [["A2"]]
score =  Score(notes, length)

data = {'pitch':1.,
        'time_evo':x,
        'azimuth':(x*0.5+0.25) % 1,
        'polar':0.5,
        'volume':y**0.7}

# set up source
sources = Objects(data.keys())
sources.fromdict(data)
sources.apply_mapping_functions()

soni = Sonification(score, sources, generator, system)
soni.render()
soni.notebook_display()

### <u>Example 3</u> &nbsp; **Filter Cutoff Mapping**

In [ ]:
generator = Synthesizer()
generator.modify_preset({'filter':'on'})

In [ ]:
notes = [["C2","G2","C3","G3"]]
score =  Score(notes, length)

data = {'pitch':[0,1,2,3],
        'time_evo':[x]*4,
        'azimuth':[(x*0.5+0.25) % 1]*4,
        'polar':[0.5]*4,
        'cutoff':[y**0.8]*4}

# set up source
sources = Objects(data.keys())
sources.fromdict(data)
sources.apply_mapping_functions()

soni = Sonification(score, sources, generator, system)
soni.render()
soni.notebook_display()

In [ ]:
generator = Synthesizer()
generator.load_preset('windy')

In [ ]:
data = {'pitch':[0,1,2,3],
        'time_evo':[x],
        'azimuth':[(x*0.5+0.25) % 1],
        'polar':[0.5],
        'cutoff':[y**0.8]}
sources = Objects(data.keys())
sources.fromdict(data)
sources.apply_mapping_functions()

soni = Sonification(score, sources, generator, system)
soni.render()
soni.notebook_display()